#### import

In [271]:
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from matplotlib import font_manager
import sklearn
import pprint
import re
from math import sqrt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import RFE, SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MultiLabelBinarizer
from hyperopt import fmin, tpe, hp, Trials
from bayes_opt import BayesianOptimization
from haversine import haversine, haversine_vector
from geopy.geocoders import Nominatim
import ast
import warnings

warnings.filterwarnings('ignore')

font_fname = 'C:/Windows/Fonts/malgun.ttf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
plt.rcParams["axes.unicode_minus"] = False

%matplotlib inline

In [272]:
df = pd.read_csv('청약홈_합본.csv', encoding='cp949').drop('Unnamed: 0', axis=1)
df.shape

(3347, 36)

In [273]:
df.keys()

Index(['아파트명', '면적', '법정동주소', '도로명주소', '위도', '경도', '세대수', '임대세대수', '영구임대세대수',
       '국민임대세대수', '민간임대세대수', '공공임대세대수', '장기임대세대수', '기타임대세대수', '최고층', '최저층',
       '최대공급면적', '최소공급면적', '총아파트동수', '용적률', '건폐율', '주차대수', '세대평균_주차대수', '건설사',
       '공급면적', '전용면적', '전용율', '방수', '욕실수', '해당면적_세대수', '현관구조', '가격',
       '초등학교_학군정보', '초등학교_설립정보', '입주예정일', '공급액(만원)'],
      dtype='object')

In [274]:
# 1차 중복 제거
df.drop_duplicates(inplace=True)

In [275]:
df.shape

(3345, 36)

### 중복컬럼, 회귀 제외 컬럼 제거

- 면적: 공급면적과 중복
- XX임대세대수: 임대세대수의 상세값
- 현관구조: 대부분 계단형
- 초등학교: 너무 많은 카테고리를 가지는 컬럼
- 입주예정일: 입주예정년으로 변경하여 카테고리를 줄임

In [276]:
df['현관구조'].value_counts()

계단식    3194
혼합식      63
복도식      19
Name: 현관구조, dtype: int64

In [277]:
len(df['초등학교_학군정보'].unique())

322

In [278]:
df['입주예정연도'] = df['입주예정일'].apply(lambda x: str(x).replace("-","")[0:4])

In [279]:
drop_col_list = ['면적','영구임대세대수','국민임대세대수', '민간임대세대수', '공공임대세대수', '장기임대세대수', 
                 '기타임대세대수','현관구조','초등학교_학군정보', '초등학교_설립정보']
df.drop(drop_col_list, axis=1).shape

(3345, 27)

In [280]:
df['location'] = tuple(zip(df['위도'], df['경도']))
apartment_location = list(df['location'])

In [281]:
df2 = df.drop(drop_col_list, axis=1)

# 변수 추가

- 대형건설사
- 가까운 지하철 역과 그 거리
- 주변 의료기관 개수
- 공원
- 대학
- 광역/기초
- 상권

### 대형 건설사

- 대형 건설사가 참여한 아파트를 구분

In [282]:
# 숫자, ., 탭, ㈜, 주식회사 등 제거 
df2['건설사'] = df2['건설사'].str.replace("[0-9.\s]|\t|\(주\)|㈜|주\)|주식회사", "", regex=True)

In [283]:
# 건설사 이름 통일
def replace_firm_name(firm):
    if '현대산업개발' in firm:
        firm = firm.replace('HDC현대산업개발', '현대산업개발')
        firm = firm.replace('에이치디씨현대산업개발', '현대산업개발')
        firm = firm.replace('현대산업개발', 'HDC현대산업개발')
    if '디엘이앤씨' in firm:
        firm = firm.replace('디엘이앤씨', 'DL이앤씨') 
    if '지에스건설' in firm:
        firm = firm.replace('지에스건설', 'GS건설')
    if '에스케이에코플랜트' in firm:
        firm = firm.replace('에스케이에코플랜트', 'SK에코플랜트')
    if '에스케이건설' in firm:
        firm = firm.replace('에스케이건설', 'SK에코플랜트') # 사명 변경
    if '포스코건설' in firm:
        firm = firm.replace('포스코건설', '포스코이앤씨')   # 사명 변경
    
    return firm

df2['건설사'] = df2['건설사'].apply(replace_firm_name)

'삼성물산', '현대건설', 'DL이앤씨', '포스코이앤씨', 'GS건설', '대우건설', '현대엔지니어링', '롯데건설', 'SK에코플랜트', 'HDC현대산업개발'

In [284]:
# 시공능력 기준 상위 10개의 건설사
대형건설사 = ['삼성물산', '현대건설', 'DL이앤씨', '포스코이앤씨', 'GS건설', '대우건설', '현대엔지니어링', '롯데건설', 'SK에코플랜트', 'HDC현대산업개발']
df2['대형건설사'] = df2['건설사'].apply(lambda x: True if any(keyword in x for keyword in 대형건설사) else False)

### 지하철 역 좌표 전처리

도시철도 역사정보: https://data.kric.go.kr/rips/M_01_01/detail.do;jsessionid=1jTd7vFOC+oGNYyEJtzQH6i1?id=32&lcd=A

서울 지하철역 좌표: https://observablehq.com/@taekie/seoul_subway_station_coordinate

In [285]:
all_rail = pd.read_excel("전체_도시철도역사정보_20230630.xlsx")

In [286]:
# 서울, 인천, 경기만 필터링
cities_to_remove = ['대구', '부산', '대전', '광주광역시', '울산'] # '광주'만 넣을 경우 '경기도 광주' 포함

# 기관명 또는 주소에 필터링 적용
greater_seoul_rail = ~(all_rail['운영기관명'].str.contains('|'.join(cities_to_remove)) | all_rail['역사도로명주소'].str.contains('|'.join(cities_to_remove)))
rail = all_rail[greater_seoul_rail]

In [287]:
# 괄호와 괄호글, 공백, 탭, .,  줄바꿈 등 제거
rail['역사명'] = rail['역사명'].str.replace(r"\s|\n|\([^()]*\)|[.·]", "", regex=True)

# 역으로 안끝나면 역을 붙여줌
rail['역사명'] = rail['역사명'].apply(lambda x: x + '역' if not x.endswith('역') else x)

In [288]:
rail = rail[['역번호', '역사명', '노선번호', '노선명', '환승역구분', '환승노선명', '역위도', '역경도', '역사도로명주소']]

In [289]:
rail = rail[rail['노선명'] != '자기부상철도'] # 인천공항 자기부상열차는 무기한 운행 중단

In [290]:
# station 데이터로 보완
station = pd.read_csv("station_coordinate.csv")
station['name'] = station['name'] + '역'
station['name'] = station['name'].str.replace("[.,]", "", regex=True)
station_name = set(station['name'].unique())

rail_name = set(rail['역사명'].unique())

In [291]:
list(station_name - rail_name)

['원곡역', '이수역']

In [292]:
# 총신대입구(이수)역은 4호선과 7호선 2개 역
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '4호선'), '역위도'] = station[station['name'] == '총신대입구역']['lat'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '4호선'), '역경도'] = station[station['name'] == '총신대입구역']['lng'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '7호선'), '역위도'] = station[station['name'] == '이수역']['lat'].values
rail.loc[(rail['역사명'] == '총신대입구역') & (rail['노선명'] == '7호선'), '역경도'] = station[station['name'] == '이수역']['lng'].values

In [293]:
# '원곡역'은 '시우역'으로 역사명이 변경되었음
rail[rail['역사명'] == '시우역']

,역번호,역사명,노선번호,노선명,환승역구분,환승노선명,역위도,역경도,역사도로명주소
1041,4814,시우역,I41WS,서해선,일반역,NaN,37.313767,126.798303,경기도 안산시 단원구 동산로 지하 50


In [294]:
# 하지만 nan값임
station[station['name'] == '원곡역']

,line,name,code,lat,lng
602,서해선,원곡역,4814.0,NaN,NaN


In [295]:
merged_rail = rail.merge(station[['name', 'lat', 'lng']], left_on = '역사명', right_on='name', how='left')

merged_rail['역위도'] = merged_rail['역위도'].fillna(merged_rail['lat'])
merged_rail['역경도'] = merged_rail['역경도'].fillna(merged_rail['lng'])

merged_rail.drop(['name', 'lat', 'lng'], axis=1, inplace=True)
merged_rail = merged_rail.drop_duplicates()

In [340]:
blank_df = merged_rail[merged_rail['역위도'].isna() | merged_rail['역경도'].isna()]
blank_df.shape

(0, 11)

In [296]:
# 나머지 25개 채워넣기, Nominatim은 지번주소로만 위경도를 추가할 수 있음으로 수정

address_updates = [
    ('산곡역', '인천광역시 부평구 산곡동 10-32'),
    ('오남역', '경기도 남양주시 오남읍 진건오남로 929'),
    ('강일역', '서울특별시 강동구 강일동 산22-14'),
    ('미사역', '경기도 하남시 망월동 109-8'),
    ('하남풍산역', '경기도 하남시 덕풍동 727-3'),
    ('하남시청역', '경기도 하남시 신장동 510-2'),
    ('하남검단산역', '경기도 하남시 창우동 526'),
    ('남위례역', '경기도 성남시 수정구 복정동 57'),
    ('삼전역', '서울특별시 송파구 잠실동 347'),
    ('석촌고분역', '서울특별시 송파구 석촌동 157'),
    ('송파나루역', '서울특별시 송파구 송파동 3'),
    ('한성백제역', '서울특별시 송파구 방이동 88-17'),
    ('둔촌오륜역', '서울특별시 강동구 강동대로 303'),
    ('중앙보훈병원역', '서울특별시 강동구 둔촌동 8-1'),
    ('구로역', '서울특별시 구로구 구로동 585-3'),
    ('구리역', '경기도 구리시 인창동 244-2'),
    ('걸포북변역', '경기도 김포시 북변동 135-10'),
    ('사우역', '경기도 김포시 사우동 854'),
    ('풍무역', '경기도 김포시 김포대로 710'),
    ('화전역', '경기도 고양시 덕양구 화전동 183-10'),
    ('화정역', '경기도 고양시 덕양구 화정동 1098'),
    ('장기역', '경기도 김포시 장기동 1791'),
    ('운양역', '경기도 김포시 운양동 1403')
]

In [297]:
for station_name, address in address_updates:
    merged_rail.loc[merged_rail['역사명'] == station_name, '역사도로명주소'] = address

In [299]:
# 주소를 기반으로 위경도를 채워넣기
def geocoding(address):
    geo_local = Nominatim(user_agent='South Korea')
    location = geo_local.geocode(address)
    if location is not None and location.latitude is not None and location.longitude is not None:
        geo = (location.latitude, location.longitude)
        return geo
    else:
        return (0, 0)
    
    
coordinates_list = []
for address in blank_df['역사도로명주소']:
    coordinates = geocoding(address)
    coordinates_list.append(coordinates)

# 리스트를 데이터프레임에 열로 추가
blank_df['역위도'], blank_df['역경도'] = zip(*coordinates_list)

In [300]:
for index, data in merged_rail.iterrows():
    if data['역사명'] in blank_df['역사명'].values:
        new_info = blank_df.loc[blank_df['역사명'] == data['역사명']]
        merged_rail.at[index, '역위도'] = new_info['역위도'].values[0]
        merged_rail.at[index, '역경도'] = new_info['역경도'].values[0]

In [301]:
duplicated = merged_rail[merged_rail.duplicated(subset=['역위도', '역경도'], keep=False) & 
                         ~merged_rail.duplicated(subset=['역위도', '역경도', '역사명'], keep=False)
                        ]
duplicated_sorted = duplicated.sort_values(by=['역위도'])
duplicated_sorted

,역번호,역사명,노선번호,노선명,환승역구분,환승노선명,역위도,역경도,역사도로명주소
716,1701,구로역,I4101,경부선,일반역,NaN,37.503178,126.882037,서울특별시 구로구 구로동 585-3
814,1205,구리역,I4108,경의중앙선,일반역,NaN,37.503178,126.882037,경기도 구리시 인창동 244-2
784,1268,화전역,I4108,경의중앙선,일반역,NaN,37.637837,126.832503,경기도 고양시 덕양구 화전동 183-10
1016,1952,화정역,I4106,일산선,일반역,NaN,37.637837,126.832503,경기도 고양시 덕양구 화정동 1098


In [336]:
names = ['구로역','구리역','화전역','화정역']
for name in names:
    address = merged_rail[merged_rail['역사명'] == name]['역사도로명주소'].iloc[0]
    coordinates = geocoding(address)
    merged_rail.loc[merged_rail['역사명'] == name, '역위도'] = coordinates[0]
    merged_rail.loc[merged_rail['역사명'] == name, '역경도'] = coordinates[1]

In [303]:
merged_rail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766 entries, 0 to 1037
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   역번호      766 non-null    object 
 1   역사명      766 non-null    object 
 2   노선번호     766 non-null    object 
 3   노선명      766 non-null    object 
 4   환승역구분    766 non-null    object 
 5   환승노선명    280 non-null    object 
 6   역위도      766 non-null    float64
 7   역경도      766 non-null    float64
 8   역사도로명주소  766 non-null    object 
dtypes: float64(2), object(7)
memory usage: 76.0+ KB


In [304]:
merged_rail['노선명'].unique()

array(['인천지하철 1호선', '인천지하철 2호선', '도시철도 7호선', '에버라인', '인천국제공항선', '우이신설선',
       '신분당선', '수도권 경량도시철도 신림선', '김포골드라인', '진접선', '1호선', '2호선', '3호선',
       '4호선', '5호선', '6호선', '7호선', '8호선', '수도권  도시철도 9호선', '의정부', '경강선',
       '경부선', '경원선', '경의중앙선', '경인선', '경춘선', '분당선', '수인선', '안산과천선', '일산선',
       '서해선'], dtype=object)

In [305]:
# 명칭 통일, 위경도 바뀐 데이터 수정
station_updates = [
    ('도시철도 7호선', '7호선'),
    ('수도권  도시철도 9호선', '9호선'),
    ('의정부', '의정부선'),
    ('일산선', '3호선'),
    ('수도권 경량도시철도 신림선', '신림선'),
    ('경부선', '1호선'),
    ('경인선', '1호선'),
    ('경원선', '1호선'),
    ('안산과천선', '4호선')
]

# 명칭
for station_name, station_line in station_updates:
    merged_rail.loc[merged_rail['노선명'] == station_name, '노선명'] = station_line

# 위경도
errored = merged_rail['노선명'] == '신분당선'
merged_rail.loc[errored, ['역위도', '역경도']] = merged_rail.loc[errored, ['역경도', '역위도']].values

In [306]:
merged_rail['location'] = tuple(zip(merged_rail['역위도'], merged_rail['역경도']))
merged_rail['subway'] = merged_rail['노선명'] + ' ' + merged_rail['역사명']

In [307]:
merged_rail.to_csv("subway.csv", encoding='UTF-8', index=False)

### 수집한 데이터 오류 수정

- 청약 신도시 사업이 끝난 후 주소를 정상 발급된 지번(도로명)주소로 넣기
- 결측값을 좀 더 채워넣었음

In [351]:
fixed = pd.read_csv("fixed_20230829.csv")

In [341]:
fixed.shape

(3133, 26)

In [345]:
fixed.columns

Index(['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사'],
      dtype='object')

지하철역

In [352]:
fixed['location'] = tuple(zip(fixed['위도'], fixed['경도']))
fixed_location = list(fixed['location'])

subway_dict = dict(zip(merged_rail['subway'], merged_rail['location']))
subway_location = list(subway_dict.values())
to_subway = haversine_vector(subway_location, fixed_location, comb=True)

close_subway = []
close_distance = []
for i in range(len(to_subway)):
    subway_index = int(np.argmin(to_subway[i]))
    closest_subway = list(subway_dict.keys())[subway_index]
    closest_distance = to_subway[i][subway_index]
    close_subway.append(closest_subway)
    close_distance.append(closest_distance)
    
fixed.loc[:, '지하철역'] = close_subway
fixed.loc[:, '지하철역_거리'] = close_distance
fixed.loc[:, '지하철역_거리'] = fixed['지하철역_거리'].round(4)

In [353]:
fixed['지하철역']

0            5호선 둔촌동역
1            5호선 둔촌동역
2            5호선 둔촌동역
3            5호선 둔촌동역
4            5호선 둔촌동역
            ...      
3322    인천지하철 2호선 독정역
3323    인천지하철 2호선 독정역
3324    인천지하철 2호선 독정역
3325    인천지하철 2호선 독정역
3326    인천지하철 2호선 독정역
Name: 지하철역, Length: 3327, dtype: object

In [354]:
rail_list = merged_rail[['역사명', '노선명']]
grouped_rail = rail_list.groupby('역사명')['노선명'].apply(list).reset_index()
grouped_rail.rename(columns={'노선명': '노선명_리스트'}, inplace=True)

In [355]:
fixed['지하철역'] = fixed['지하철역'].apply(lambda x: x.split(" ")[-1])
apartment_rail = pd.merge(fixed, grouped_rail, left_on='지하철역', right_on='역사명', how='inner')

In [356]:
fixed['지하철역']

0       둔촌동역
1       둔촌동역
2       둔촌동역
3       둔촌동역
4       둔촌동역
        ... 
3322     독정역
3323     독정역
3324     독정역
3325     독정역
3326     독정역
Name: 지하철역, Length: 3327, dtype: object

In [349]:
apartment_rail['지하철역']

0         둔촌동역
1         둔촌동역
2         둔촌동역
3         둔촌동역
4         둔촌동역
         ...  
3322    을지로3가역
3323    을지로3가역
3324    을지로3가역
3325    을지로3가역
3326    을지로3가역
Name: 지하철역, Length: 3327, dtype: object

병원

In [313]:
hospital = pd.read_csv("hospital.csv")
hospital = hospital[hospital['location'] != '(0.0, 0.0)']
hospital.loc[:, 'location'] = hospital['location'].apply(ast.literal_eval)

In [314]:
hospital_location_1 = list(hospital[hospital['종별코드명'] == '1차']['location'].unique())
hospital_location_2 = list(hospital[hospital['종별코드명'] == '2차']['location'].unique())
hospital_location_3 = list(hospital[hospital['종별코드명'] == '3차']['location'].unique())

In [315]:
distance_to_hospitals = haversine_vector(hospital_location_1, fixed_location, comb=True)

num_close_hospital_1 = []
for distance in distance_to_hospitals:
    number = np.sum(distance <= 1)
    num_close_hospital_1.append(number)
apartment_rail['1차병원'] = num_close_hospital_1


distance_to_hospitals = haversine_vector(hospital_location_2, fixed_location, comb=True)

num_close_hospital_2 = []
for distance in distance_to_hospitals:
    number = np.sum(distance <= 2)
    num_close_hospital_2.append(number)

apartment_rail['2차병원'] = num_close_hospital_2


distance_to_hospitals = haversine_vector(hospital_location_3, fixed_location, comb=True)

num_close_hospital_3 = []
for distance in distance_to_hospitals:
    number = np.sum(distance <= 3)
    num_close_hospital_3.append(number)
apartment_rail['3차병원'] = num_close_hospital_3

#### 공원

전국도시공원정보표준데이터: https://www.data.go.kr/data/15012890/standard.do

In [316]:
park = pd.read_csv('park.csv', encoding='euc-kr')
park_df = park[~park['공원구분'].isin(['묘지공원', '도시농업공원', '가로공원', '공공공지', '기타', '기타공원'])]
park_df.loc[park_df['공원구분'] == '어린인공원', '공원구분'] = '어린이공원'
park_df.loc[park_df['공원구분'] == '도시자연공원구역', '공원구분'] = '도시자연공원'
park_df.loc[park_df['공원구분'] == '도시자연공원구역', '공원구분'] = '도시자연공원'
park_df = park_df[park_df['제공기관명'].str.contains('서울|경기|인천')]
park_df = park_df.dropna(subset=['위도', '경도'])
park_df['location'] = tuple(zip(park_df['위도'], park_df['경도']))
park_df = park_df[['공원명', 'location']]

In [317]:
location = list(apartment_rail['location'])
park_location = list(park_df['location'])

distance_to_park = haversine_vector(park_location, location, comb=True)

num_close_park = []
for distance in distance_to_park:
    number = np.sum(distance <= 1)
    num_close_park.append(number)
    
apartment_rail['공원'] = num_close_park

#### 대학

교육부_전국대학교개황: https://www.data.go.kr/data/15100330/fileData.do

In [318]:
university = pd.read_csv("university.csv")
university_df = university[university['학교구분'].isin(['대학', '전문대학'])]

In [319]:
university['location'] = tuple(zip(university['Latitude'], university['Longitude']))
university_location = list(university['location'])
distance_to_univ = haversine_vector(university_location, location, comb=True)

num_close_univ = []
for distance in distance_to_univ:
    number = np.sum(distance <= 1)
    num_close_univ.append(number)
    
apartment_rail['대학'] = num_close_univ

In [320]:
apartment_rail[apartment_rail['공원'].isna() | apartment_rail['대학'].isna()]

,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,location,지하철역,지하철역_거리,역사명,노선명_리스트,1차병원,2차병원,3차병원,공원,대학


#### 광역 / 기초 변수 추가 & 상권 변수 추가 

In [321]:
apartment_rail.loc[:, '법정동주소'] = apartment_rail['법정동주소'].str.replace(r"\t", "", regex=True)
apartment_rail['광역'] = apartment_rail['법정동주소'].apply(lambda x: x.split(" ")[0])
apartment_rail['광역'].value_counts()

경기도      1931
인천광역시     484
서울특별시     383
서울시       197
인천시       131
안양동         4
광주시         3
Name: 광역, dtype: int64

In [322]:
apartment_rail.loc[apartment_rail['광역'] == '안양동', '법정동주소'] = '경기도 안양시 만안구 안양동 전파로61번길 20'
apartment_rail.loc[apartment_rail['광역'] == '광주시', '법정동주소'] = '경기도 광주시 탄벌동 532-2번지'

apartment_rail['광역'] = apartment_rail['법정동주소'].apply(lambda x: x.split(" ")[0])
apartment_rail.loc[(apartment_rail['광역'] == '서울시') | (apartment_rail['광역'] == '서울특별시'), '광역'] = '서울'
apartment_rail.loc[(apartment_rail['광역'] == '인천시') | (apartment_rail['광역'] == '인천광역시'), '광역'] = '인천'
apartment_rail.loc[(apartment_rail['광역'] == '경기도'), '광역'] = '경기'

apartment_rail['광역'].value_counts()

경기    1938
인천     615
서울     580
Name: 광역, dtype: int64

In [323]:
apartment_rail['기초'] = apartment_rail['법정동주소'].apply(lambda x: x.split(" ")[1])
apartment_rail['기초'].unique()

array(['강동구', '강남구', '성남시', '수원시', '광명시', '연수구', '동대문구', '서초구', '부평구',
       '안양시', '성북구', '의왕시', '서구', '평택시', '안성시', '용인시', '광주시', '중구',
       '미추홀구', '남동구', '화성시', '이천시', '의정부시', '부천시', '파주시', '구리시', '남양주시',
       '구로구', '강북구', '계양구', '오산시', '도봉구', '양주시', '시흥시', '과천시', '포천시',
       '동작구', '용인처인구', '안산시', '강서구', '은평구', '강화군', '동구', '송파구', '고양시',
       '노원구', '가평군', '동두천시', '광진구', '중랑구', '종로구', '하남시', '양평군', '김포시',
       '영등포구', '여주시', '관악구', '양천구', '고양덕양구', '연천군', '성동구', '서대문구', '군포시',
       '성남수정구', '용산구'], dtype=object)

In [324]:
apartment_rail.loc[apartment_rail['기초'] == '용인처인구', '기초'] = '용인시'
apartment_rail.loc[apartment_rail['기초'] == '성남수정구', '기초'] = '성남시'
apartment_rail.loc[apartment_rail['기초'] == '고양덕양구', '기초'] = '고양시'

In [325]:
apartment_rail['기초'].unique()

array(['강동구', '강남구', '성남시', '수원시', '광명시', '연수구', '동대문구', '서초구', '부평구',
       '안양시', '성북구', '의왕시', '서구', '평택시', '안성시', '용인시', '광주시', '중구',
       '미추홀구', '남동구', '화성시', '이천시', '의정부시', '부천시', '파주시', '구리시', '남양주시',
       '구로구', '강북구', '계양구', '오산시', '도봉구', '양주시', '시흥시', '과천시', '포천시',
       '동작구', '안산시', '강서구', '은평구', '강화군', '동구', '송파구', '고양시', '노원구',
       '가평군', '동두천시', '광진구', '중랑구', '종로구', '하남시', '양평군', '김포시', '영등포구',
       '여주시', '관악구', '양천구', '연천군', '성동구', '서대문구', '군포시', '용산구'],
      dtype=object)

#### 상권_변수

In [326]:
area = pd.read_csv("상권_변수.csv")
area.head()

,광역시도,시군구,소매,음식,교육,장례식장,보건의료,유원지오락,총인구수
0,서울,종로구,"9,208","6,184",975,11,198,246,"141,223"
1,서울,중구,"12,053","5,521",587,18,253,204,"120,317"
2,서울,용산구,"9,175","4,830",856,9,137,176,740
3,서울,성동구,"9,516","4,310","1,034",7,171,276,"280,707"
4,서울,광진구,"10,989","4,955","1,016",7,203,556,"337,258"


In [327]:
merged = pd.merge(apartment_rail, area, left_on=['광역', '기초'], right_on=['광역시도', '시군구'], how='inner')

In [328]:
merged = merged[['아파트명', '법정동주소', '위도', '경도', '세대수', '임대세대수', '최고층', '최저층', '최대공급면적',
       '최소공급면적', '총아파트동수', '용적률', '건폐율', '세대평균_주차대수', '공급면적', '전용면적', '전용율',
       '방수', '욕실수', '현관구조', '입주예정연도', '공급액(만원)', '대형건설사', '지하철역', '지하철역_거리', 
       '역사명', '노선명_리스트', '1차병원', '2차병원', '3차병원', '공원', '대학', '광역', '기초', 
       '소매', '음식', '교육', '장례식장', '보건의료', '유원지오락', '총인구수']]

##### 추가적인 전처리

- 용적률과 건폐율이 바뀜
- 세대평균 주차대수가 잘못된 값 수정

In [329]:
merged.loc[merged['건폐율'] > 100, ['용적률','건폐율']] = merged.loc[merged['건폐율'] > 100, ['건폐율', '용적률']].values
merged[merged['아파트명'] == '가평 센트럴파크 더 스카이'][['아파트명', '용적률', '건폐율']] 

,아파트명,용적률,건폐율
2852,가평 센트럴파크 더 스카이,482.0,66.0
2853,가평 센트럴파크 더 스카이,482.0,66.0
2854,가평 센트럴파크 더 스카이,482.0,66.0
2855,가평 센트럴파크 더 스카이,482.0,66.0
2856,가평 센트럴파크 더 스카이,482.0,66.0
2857,가평 센트럴파크 더 스카이,482.0,66.0
2858,가평 센트럴파크 더 스카이,482.0,66.0
2859,가평 센트럴파크 더 스카이,482.0,66.0


In [330]:
merged[merged['세대평균_주차대수']>=3][['아파트명', '세대평균_주차대수']]

,아파트명,세대평균_주차대수
92,고덕 아르테스 미소지움,3.3
93,고덕 아르테스 미소지움,3.3
94,고덕 아르테스 미소지움,3.3
703,안양 광신프로그레스 리버뷰,6.3
704,안양 광신프로그레스 리버뷰,6.3
705,안양 광신프로그레스 리버뷰,6.3
706,안양 광신프로그레스 리버뷰,6.3
707,평촌 트리지아,3.3
708,평촌 트리지아,3.3
709,평촌 트리지아,3.3


In [331]:
merged.loc[merged['아파트명'] == '대곡역 롯데캐슬 엘클라씨', '세대평균_주차대수'] = 1.2
merged.loc[merged['아파트명'] == '고덕 아르테스 미소지움', '세대평균_주차대수'] = 1.2
merged.loc[merged['아파트명'] == '안양 광신프로그레스 리버뷰', '세대평균_주차대수'] = 1.17
merged.loc[merged['아파트명'] == '평촌 트리지아', '세대평균_주차대수'] = 1.26
merged.loc[merged['아파트명'] == '자양 하늘채 베르', '세대평균_주차대수'] = 0.98

In [332]:
merged.to_csv("apartment_20230823.csv", encoding='UTF-8', index=False)